# Building Correspondence Matrix for Footprint Calculation

This Jupyter notebook demonstrates how to calculate the household Carbon Footprint, using Germany as an example. Prior to the calculation, the datasets of national household consumption and consumer expenditure survey are collected from Eurostat. The matrices of product taxes and trade margins are collected from EXIOBASE Supply-Use Tables (SUTs).

This work is adapted from Hardadi, et al. (2020).

# 1. Step: Import of required libraries

In this step all the libraries needed within the script are imported

In [1]:
import xlrd, xlsxwriter
import numpy as np
import scipy.io
import scipy

# 2. Step: Import Exiobase 3.4

Now, Exiobase 3.4 will be imported. This includes the following:

L-Matrix containing the Leontief-Inverse

S-Matrix containing the emissions

Y-Matrix containing the final demands

FDE-Matrix containing the direct emissions caused by the final demands

In [2]:
Filestring_Matlab_in = "C://Users//Gilang Hardadi//Documents//EXIOBASE_2//EXIOBASE3_10_ITC_pxp.mat"
#This variable is the location of the previously parsed EXIOBASE dataset

print('Import L-Matrix (Leontief-Inverse)')
MRIO_L = scipy.io.loadmat(Filestring_Matlab_in)['EB3_L_ITC']

print('Import S-Matrix (Emissions).')
MRIO_S = scipy.io.loadmat(Filestring_Matlab_in)['EB3_S_ITC']

print('Import Y-Matrix (Final Demands)')
MRIO_Y = scipy.io.loadmat(Filestring_Matlab_in)['EB3_Y']

print('Import FDE-Matrix (Direct Emissions from Final Demand).')
MRIO_FDE = scipy.io.loadmat(Filestring_Matlab_in)['EB3_FinalDemand_Emissions']

print('Import the Names of Industry Sectors.')
MRIO_Prod = scipy.io.loadmat(Filestring_Matlab_in)['EB3_ProductNames200']

print('Import the Names of Extension Codes.')
MRIO_Ext = scipy.io.loadmat(Filestring_Matlab_in)['EB3_Extensions']

print('Import the Names of Regions.')
MRIO_Reg = scipy.io.loadmat(Filestring_Matlab_in)['EB3_RegionList']

NoofCountries  = 49
NoofProducts   = 200
NoofIndustries = 163
NoofFinDemands = 7

Population = 80.645605 #million people
Country_ID = 5 #Index position for Germany in EXIOBASE

Import L-Matrix (Leontief-Inverse)
Import S-Matrix (Emissions).
Import Y-Matrix (Final Demands)
Import FDE-Matrix (Direct Emissions from Final Demand).
Import the Names of Industry Sectors.
Import the Names of Extension Codes.
Import the Names of Regions.


# 3. Step: Import characterisation factors

In order to calculate the environmental footprints, characterisation factors are needed
to convert the embodied emissions for each type of Greenhouse Gases (GHGs) to midpoint indicators

In [3]:
ImpactFile  = xlrd.open_workbook('Characterization_EB36.xlsx')
ImpactSheet = ImpactFile.sheet_by_name('Emissions')
ImpactCategory_Names = []
for m in range(0,36):
    ImpactCategory_Names.append(ImpactSheet.cell_value(0,m+1))
    
MRIO_Char = np.zeros((36,1707))
for m in range(0,36):
    for n in range(0,1707):
        MRIO_Char[m,n] = ImpactSheet.cell_value(n+1,m+1)

print(ImpactCategory_Names)

['CO2 - GWP100                               ', 'CH4 - GWP100                               ', 'N2O - GWP100                               ', 'SF6 - GWP100                               ', 'Global Warming Potential 100               ', 'Land use - crops                           ', 'Land use - pasture                         ', 'Land use - forest                          ', 'Land use                                   ', 'Total Emission relevant energy use         ', 'Total Energy inputs from nature            ', 'Total Energy supply                        ', 'Total Energy Use                           ', 'Domestic extraction  - primary crops       ', 'Domestic extraction  - crops residues      ', 'Domestic extraction  - fodder crops        ', 'Domestic extraction - grazing              ', 'Domestic extraction - wood                 ', 'Domestic extraction - aquatic animals      ', 'Domestic extraction - metal ores           ', 'Domestic extraction - nonmetallic minerals ', 'Domestic ex

# 4. Step: Import initial correspondence matrix

Initial correspondence matrix is a binary matrix which maps the consumption categories from one classification (in this case COICOP) to another classification (EXIOBASE). In this matrix, 1 represents that the initial category (COICOP) is being assigned to the target category (EXIOBASE), while 0 means that the initial category is not mapped to the target category. This matrix is built according to the United Nation classification.

In [5]:
CorrespondenceMatrix_file = xlrd.open_workbook('Correspondence_Matrix.xlsx')
CorrespondenceMatrix_sheet = CorrespondenceMatrix_file.sheet_by_name('Matrix')

# Use this correspondence matrix as iteration point for obtaining matrix (matrices) transforming 
# expenditures in CES sectors into EXIOBASE sectors.
CorrespondenceMatrixInitial = np.zeros((200,104))
for m in range(0,200):
    for n in range(0,104):
        CorrespondenceMatrixInitial[m,n] = CorrespondenceMatrix_sheet.cell_value(m+1,n+1)       

print(CorrespondenceMatrixInitial)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]]


# 5. Step: Import data on taxes on products and trade and transport margins

This data is taken from EXIOBASE SUTs, which is necessary to later convert the consumption data in purchaser's price to basic price.

In [6]:
TaxTradeMargin_file = xlrd.open_workbook('DE_2010.xlsx')
TradeMargins_sheet = TaxTradeMargin_file.sheet_by_name('trade_margins_init')
TransMargins_sheet = TaxTradeMargin_file.sheet_by_name('transport_margins_init')
ProductTaxes_sheet = TaxTradeMargin_file.sheet_by_name('product_taxes_init')

TradeMargins = np.zeros((200))
TransMargins = np.zeros((200))
ProductTaxes = np.zeros((200))
for m in range(0,200):
    TradeMargins[m] = TradeMargins_sheet.cell_value(m+14,168)/1e6 #to convert the unit from Euro to MEuro
    TransMargins[m] = TransMargins_sheet.cell_value(m+14,168)/1e6
    ProductTaxes[m] = ProductTaxes_sheet.cell_value(m+14,168)/1e6


# 6. Step: Import national household consumption data in COICOP classification

The national household consumption data in COICOP classification is collected from Eurostat database. The data is available for 42 consumption categories. In this datasheet, category IDs are also assigned for each consumption category.

In [7]:
HH_expenditure_file = xlrd.open_workbook('nama_10_co3_p3.xls')
HH_expenditure_sheet = HH_expenditure_file.sheet_by_name('Data')

HHE_CategoryNames = []
HHE_CategoryAggID = np.zeros(42, dtype=int)
HH_Expenditure_DE = np.zeros(42)
for m in range(0,42):
    HHE_CategoryNames.append(HH_expenditure_sheet.cell_value(10,m+1))
    HHE_CategoryAggID[m] = HH_expenditure_sheet.cell_value(9,m+1)
    try:
        HH_Expenditure_DE[m] = HH_expenditure_sheet.cell_value(15,m+1)
    except ValueError:
        HH_Expenditure_DE[m] = 0

print(HHE_CategoryNames)
print(HHE_CategoryAggID)
print(HH_Expenditure_DE)

['Food', 'Non-alcoholic beverages', 'Alcoholic beverages', 'Tobacco', 'Narcotics', 'Clothing', 'Footwear', 'Actual rentals for housing', 'Imputed rentals for housing', 'Maintenance and repair of the dwelling', 'Water supply and miscellaneous services relating to the dwelling', 'Electricity, gas and other fuels', 'Furniture and furnishings, carpets and other floor coverings', 'Household textiles', 'Household appliances', 'Glassware, tableware and household utensils', 'Tools and equipment for house and garden', 'Goods and services for routine household maintenance', 'Medical products, appliances and equipment', 'Out-patient services', 'Hospital services', 'Purchase of vehicles', 'Operation of personal transport equipment', 'Transport services', 'Postal services', 'Telephone and telefax equipment', 'Telephone and telefax services', 'Audio-visual, photographic and information processing equipment', 'Other major durables for recreation and culture', 'Other recreational items and equipment, 

# 7. Step: Import household expenditure survey data

The household expenditure survey data in COICOP classification is collected from Eurostat database. This data is available at a higher resolution level than the national household consumption data, for 104 consumption categories. In this datasheet, category IDs are assigned accordingly to the expenditure categories in national household consumption data.

In [8]:
CES_expenditure_file = xlrd.open_workbook('hbs_exp_t121.xls')
CES_expenditure_sheet = CES_expenditure_file.sheet_by_name('Data')

CES_CategoryNames = []
CES_CategoryAggID = np.zeros(104, dtype=int)
CES_DEexpenditure = np.zeros(104)
for m in range(0,104):
    CES_CategoryNames.append(CES_expenditure_sheet.cell_value(10,m+1))
    CES_CategoryAggID[m] = CES_expenditure_sheet.cell_value(9,m+1)
    CES_DEexpenditure[m] = CES_expenditure_sheet.cell_value(15,m+1)

print(CES_CategoryNames)
print(CES_CategoryAggID)
print(CES_DEexpenditure)

['Bread and cereals', 'Meat', 'Fish and seafood', 'Milk, cheese and eggs', 'Oils and fats', 'Fruit', 'Vegetables', 'Sugar, jam, honey, chocolate and confectionery', 'Food products n.e.c.', 'Non-alcoholic beverages', 'Alcoholic beverages', 'Tobacco', 'Narcotics', 'Clothing materials', 'Garments', 'Other articles of clothing and clothing accessories', 'Cleaning, repair and hire of clothing', 'Shoes and other footwear', 'Repair and hire of footwear', 'Actual rentals paid by tenants', 'Other actual rentals', 'Imputed rentals of owner-occupiers', 'Other imputed rentals', 'Materials for the maintenance and repair of the dwelling', 'Services for the maintenance and repair of the dwelling', 'Water supply and miscellaneous services relating to the dwelling', 'Electricity', 'Gas', 'Liquid fuels', 'Solid fuels', 'Heat energy', 'Furniture and furnishings', 'Carpets and other floor coverings', 'Repair of furniture, furnishings and floor coverings', 'Household textiles', 'Major household appliances 

# 8. Step: Build tuple matrix to aggregate expenditures with identical indices

In this step, a function is built to later aggregate expenditures with identical indices. This function is taken from PySUT library developed by Konstantin Stadler.

In [9]:
# Define ancillary functions to construct aggregation matrices

def MI_Tuple(value, Is): 
    """
    Define function for obtaining multiindex tuple from index value
    value: flattened index position, Is: Number of values for each index dimension
    Example: MI_Tuple(10, [3,4,2,6]) returns [0,0,1,4]
    MI_Tuple(138, [100,10,5]) returns [2,7,3]
    MI_Tuple is the inverse of Tuple_MI.
    """
    IsValuesRev = []
    CurrentValue = value
    for m in range(0,len(Is)):
        IsValuesRev.append(CurrentValue % Is[len(Is)-m-1])
        CurrentValue = CurrentValue // Is[len(Is)-m-1]
    return IsValuesRev[::-1]    

def Tuple_MI(Tuple, IdxLength): 
    """
    Function to return the absolution position of a multiindex when the index tuple
    and the index hierarchy and size are given.
    Example: Tuple_MI([2,7,3],[100,10,5]) returns 138
    Tuple_MI([0,0,1,4],[3,4,2,6]) returns 10
    Tuple_MI is the inverse of MI_Tuple.
    """
    # First, generate the index position offset values
    IdxShift =  IdxLength[1:] +  IdxLength[:1] # Shift 1 to left
    IdxShift[-1] = 1 # Replace lowest index by 1
    IdxShift.reverse()
    IdxPosOffset = np.cumproduct(IdxShift).tolist()
    IdxPosOffset.reverse()
    Position = np.sum([a*b for a,b in zip(Tuple,IdxPosOffset)])
    return Position

def build_Aggregation_Matrix(Position_Vector): # from PySUT
    """Turn a vector of target positions into a matrix that aggregates 
    or re-arranges rows of the table it is multiplied to from the left 
    (or columns, if multiplied transposed from the right)"""
    AM_length = Position_Vector.max() + 1 # Maximum row number of new matrix (+1 to get the right length, as 0 is the smallest target position entry.)
    AM_width  = len(Position_Vector) # Number of rows of the to-be-aggregated matrix
    Rearrange_Matrix = np.zeros((AM_length,AM_width))
    for m in range(0,len(Position_Vector)):
        Rearrange_Matrix[Position_Vector[m].item(0),m] = 1
        # place 1 in aggregation matrix at [PositionVector[m],m], so that column m is aggregated with Positionvector[m] in the aggregated matrix
    return Rearrange_Matrix

Using build_Aggregation_Matrix function, the household expenditure survey data (in 104 categories) is aggregated accordingly to the national household consumption data (in 42 categories).

In [10]:
CES_DE_agg = build_Aggregation_Matrix(CES_CategoryAggID).dot(CES_DEexpenditure)

print(CES_DE_agg)

[2960.  338.  265.  201.    0.  991.  244. 2674. 3359.  236.  679. 1640.
  526.  103.  222.   93.  152.  184.  545.  478.   76. 1195. 2404.  408.
   65.   38.  662.  416.   65.  516.  614.  500.  712.  236. 1022.  297.
  619.  150.   59. 2023.   62.  123.]


Finally, the national household consumption data is disaggregated accordingly (into 104 categories) based on the household expenditure survey data.

In [11]:
HHE_DE_dag = np.zeros((104))

for i in range(104):
    idx = CES_CategoryAggID[i]
    if CES_DE_agg[idx] != 0:
        HHE_DE_dag[i] = CES_DEexpenditure[i]/CES_DE_agg[idx]*HH_Expenditure_DE[idx]
    else:
        HHE_DE_dag[i] = 0

print(HHE_DE_dag)

[ 23870.7         28585.9          5009.9         19744.9
   3578.5         12040.6         15492.8         10398.7
   5894.          14897.          19865.          24906.
      0.           1188.39656912  51553.77497477   1697.70938446
   1641.11907164  10414.95081967    353.04918033 100157.68249813
   2025.31750187 117094.90919917   9783.09080083   5007.16949153
   5179.83050847  36564.          26766.26829268  16842.58536585
  13956.41463415   1383.7804878    5890.95121951  29699.44296578
   3414.46958175    773.08745247   6970.           8652.08108108
   3166.77477477    735.14414414   6884.           7574.
  10032.55434783   7382.44565217  14543.72293578   3998.07522936
  13037.20183486  13739.87447699   7091.54811715   2708.57740586
  14614.          51195.25188285   1520.65104603   2350.09707113
      0.           7037.34109817  42596.39267887  15191.72046589
  24686.54575707   8630.15686275   1531.15686275   7516.58823529
    835.17647059   9326.1372549     556.78431373   2657

# 9. Step: Calculate household final demand data for each EXIOBASE category in basic and purchaser's price

The MRIO_Y matrix contains national final demand data (household, government, capital formation, etc.) in 9800 region-products (200 products from 49 regions). In this step, the household final demand for German household (in basic price) is extracted and aggregated into 200 products.

In [12]:
HH_FinalDemand_bp = MRIO_Y[:,NoofFinDemands*Country_ID]
HH_FinalDemand_bp = HH_FinalDemand_bp.reshape(49,200).sum(axis=0)
print(HH_FinalDemand_bp)

[1.83012439e+00 1.22059515e+03 3.07370145e+02 1.22834965e+04
 7.61501082e+02 0.00000000e+00 2.44661457e-05 2.15723580e+03
 0.00000000e+00 0.00000000e+00 2.49159199e+03 4.15914350e+02
 3.27180124e+02 5.11700480e+01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.15481870e+03 3.84112240e+02 6.16371465e+01
 0.00000000e+00 1.76790782e+02 1.02845415e-04 2.78248336e+01
 0.00000000e+00 3.24660292e+02 1.46274558e-04 0.00000000e+00
 1.25500914e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.48667191e+02 2.49268368e+03 5.55179302e+03
 7.01694183e+03 1.17963142e+04 4.99298482e+02 1.55820270e+04
 1.74224723e+02 1.46640037e+03 5.65244341e+04 9.65656597e+03
 7.39217581e+03 3.65131686e+03 1.25984629e+04 1.88603482e+04
 6.51960189e+03 3.41460397e+03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.79152016e+03 2.46127399e+04 8.17294376e+00
 0.00000000e+00 9.099539

Then, the trade and transport margins and product taxes are added to convert the national household demand from basic price to purchaser's price.

In [13]:
HH_FinalDemand_pp = HH_FinalDemand_bp + TradeMargins + TransMargins + ProductTaxes
print(HH_FinalDemand_pp)

[1.89142148e+00 1.26706845e+03 3.21613939e+02 1.25223196e+04
 8.23570613e+02 0.00000000e+00 2.44661457e-05 2.20003671e+03
 0.00000000e+00 0.00000000e+00 2.62235062e+03 4.59566290e+02
 3.29645391e+02 5.21324128e+01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.17521055e+03 3.95117877e+02 6.27123984e+01
 0.00000000e+00 1.78294282e+02 1.02845415e-04 2.95115167e+01
 0.00000000e+00 3.39638884e+02 1.46274558e-04 0.00000000e+00
 1.35023557e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.52016006e+02 2.68900188e+03 5.79972027e+03
 7.33103620e+03 1.36730117e+04 5.14010038e+02 1.67138395e+04
 1.84103188e+02 1.54853487e+03 6.06836365e+04 1.08044907e+04
 9.77654138e+03 1.69284068e+04 1.50129826e+04 2.60286391e+04
 8.69610333e+03 3.57830317e+03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.88276498e+03 2.61814576e+04 1.58577919e+01
 0.00000000e+00 9.099539

The national household final demand data in EXIOBASE is not much different compared to the national household consumption data from Eurostat, with less than 3% difference.

In [14]:
print(HH_FinalDemand_pp.sum(axis=0))
print(HHE_DE_dag.sum(axis=0))

1280882.5257171914
1316506.0


# 10. Step: Calculate Tax and Margin Rates

The tax rates are calculated by diving the taxes by the household final demand in purchaser's price, while the margin rates are calculated by dividing the margins by the household final demand in producer's price (purchaser's price without the taxes).

In [15]:
Tax_Rate = np.zeros((200))
Mar_Rate = np.zeros((200))

for m in range(0,200):
    if HH_FinalDemand_pp[m] == 0:
        Tax_Rate[m] = 0
        Mar_Rate[m] = 0
    else:
        Tax_Rate[m] = ProductTaxes[m] / HH_FinalDemand_pp[m]
        HH_FinalDemand_pdp = HH_FinalDemand_pp[m] - ProductTaxes[m]
        if TradeMargins[m] < 0:
            Mar_Rate[m] = 0
        else:
            Mar_Rate[m] = (TradeMargins[m] + TransMargins[m]) / HH_FinalDemand_pdp
            
print(Tax_Rate)
print(Mar_Rate)

[ 0.02418355  0.03126361  0.0377614   0.0145143   0.05942865  0.
  0.          0.01714576  0.          0.          0.03937407  0.07077579
  0.00171403  0.01534796  0.          0.          0.          0.01699467
  0.02473512  0.01628453  0.          0.0081102   0.          0.05689677
  0.          0.04296694  0.          0.          0.06516156  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.0194266
  0.06995704  0.0412926   0.02760297  0.13658788  0.02815349  0.06393749
  0.05141695  0.05092256  0.06474677  0.10432469  0.23061846  0.78211018
  0.15145742  0.26338579  0.23981205  0.04090444  0.          0.
  0.          0.01317137  0.05440478  0.48111202  0.          0.
  0.28068241  0.21975195  0.          0.20178177  0.33402865  0.43671875
  0.          0.          0.31357627  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.       

# 11. Step: Calculate the correspondence matrix

The correspondence matrix (or concordance matrix) is a matrix that assigns the household consumption from home categories (in this case COICOP) to the target categories (EXIOBASE) according to the proportion of the consumption in the target categories. While the initial correspondence matrices for different countries should be similar, the correspondence matrix for each country will be different since the consumption structure for each country is different.

The correspondence matrix is calculated using RAS method (Miller and Blair, 2009), an iterative method to balance a matrix, where the desired result is that the axial sum of the matrix is identical to the target vector (in this case the household consumption in EXIOBASE categories, purchaser's price).

In [16]:
def RAS(i,j,k):
    IterationMatrix0 = i
    HH_Expenditure = j
    Final_Demand = k
    IterationMatrix1 = IterationMatrix0.dot(np.diag(np.nan_to_num(HH_Expenditure/(IterationMatrix0.sum(axis=0)))))
    counter = 0
    while True:
        IterationMatrix2 = np.diag(np.nan_to_num(Final_Demand/(IterationMatrix1.sum(axis=1)))).dot(IterationMatrix1)
        IterationMatrix1 = IterationMatrix2.dot(np.diag(np.nan_to_num(HH_Expenditure/(IterationMatrix2.sum(axis=0)))))
        counter = counter + 1
        if counter == 500:
            return IterationMatrix1

In [17]:
Correspondence_Matrix = RAS(CorrespondenceMatrixInitial, HHE_DE_dag, HH_FinalDemand_pp)

<ipython-input-16-8e7f7f2fefc2>:5: RuntimeWarning: invalid value encountered in true_divide
  IterationMatrix1 = IterationMatrix0.dot(np.diag(np.nan_to_num(HH_Expenditure/(IterationMatrix0.sum(axis=0)))))
<ipython-input-16-8e7f7f2fefc2>:8: RuntimeWarning: divide by zero encountered in true_divide
  IterationMatrix2 = np.diag(np.nan_to_num(Final_Demand/(IterationMatrix1.sum(axis=1)))).dot(IterationMatrix1)
<ipython-input-16-8e7f7f2fefc2>:8: RuntimeWarning: invalid value encountered in true_divide
  IterationMatrix2 = np.diag(np.nan_to_num(Final_Demand/(IterationMatrix1.sum(axis=1)))).dot(IterationMatrix1)
<ipython-input-16-8e7f7f2fefc2>:9: RuntimeWarning: invalid value encountered in true_divide
  IterationMatrix1 = IterationMatrix2.dot(np.diag(np.nan_to_num(HH_Expenditure/(IterationMatrix2.sum(axis=0)))))


Finally, the result from RAS procedure is being normalized to obtain the correspondence matrix.

In [18]:
Correspondence_Matrix = np.nan_to_num(Correspondence_Matrix/Correspondence_Matrix.sum(axis=0)).transpose()
print(Correspondence_Matrix)

[[8.19420616e-05 5.48932122e-02 1.39332822e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


<ipython-input-18-78a08803f9e1>:1: RuntimeWarning: invalid value encountered in true_divide
  Correspondence_Matrix = np.nan_to_num(Correspondence_Matrix/Correspondence_Matrix.sum(axis=0)).transpose()


# 12. Step: Bridge the household expenditure survey data into EXIOBASE

Initially, the expenditure survey data (in purchaser's price) is bridged into EXIOBASE categories (200 products) using the correspondence matrix.

In [19]:
# The expenditure vector is diagonalized in order to obtain the footprint value specifically for each product
Expenditure_pp = np.diag(CES_DEexpenditure).dot(Correspondence_Matrix)
print(Expenditure_pp)

[[ 0.04646115 31.12445133  7.900171   ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


Afterwards, the expenditures are converted into basic price using the tax and margin rates information. The margins are later assigned to the "Wholesale trade" and "Retail trade" categories, to take into account the emissions happening due to trade activities.

In [20]:
Expenditure_bp = np.zeros((104,200))

for m in range(0,104):
    for n in range(0,200):
        Expenditure_bp[m,n] = (1-Mar_Rate[m]) * (1-Tax_Rate[m]) * Expenditure_pp[m,n]
    Expenditure_bp[m,153] = 0.68 * (Mar_Rate[0:124] * (1-Tax_Rate[0:124])).dot(Expenditure_pp[m,0:124]) 
    Expenditure_bp[m,154] = 0.32 * (Mar_Rate[0:124] * (1-Tax_Rate[0:124])).dot(Expenditure_pp[m,0:124])
    #0.68 and 0.32 are the proportions of wholesale and retail trade in EXIOBASE for German households
    #The proportions for other countries might differ than these values.

print(Expenditure_bp)

[[ 0.04495544 30.11577167  7.64414266 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


Then the expenditure survey data in basic price is split into 9800 region-products. First of all, the regional shares of household consumption for each consumption category (200 categories) are calculated.

In [21]:
MRIO_Y_RegionalShares = np.zeros(9800)
counter = 0
for m in range(0,49):
    for n in range(0,200):
        if HH_FinalDemand_bp[n] == 0:
            MRIO_Y_RegionalShares[counter] == 0
        else:
            MRIO_Y_RegionalShares[counter] = MRIO_Y[counter,NoofFinDemands*Country_ID]/HH_FinalDemand_bp[n]
        counter = counter + 1

print(MRIO_Y_RegionalShares)

[0.         0.00465635 0.00766211 ... 0.00021632 0.         0.        ]


Assuming that the households purchase their consumption similar to the national household consumption basket, their expenditure is split into 9800 region-products based on the previously calculated regional shares.

In [23]:
EXIO_Expenditure = np.zeros((9800,104))
for m in range(0,104):
    Expenditure_Product = Expenditure_bp[m,:]
    counter = 0
    for n in range(0,49):
        for o in range(0,200):
            EXIO_Expenditure[counter,m] = Expenditure_Product[o] * MRIO_Y_RegionalShares[counter]
            counter = counter+1
            
print(EXIO_Expenditure.shape)

(9800, 104)


# 12. Step: Calculating the household Carbon Footprint

Household carbon footprint shows the total GHG emissions embodied in the household consumption, measured in CO<sub>2</sub> equivalent. The carbon footprint is calculated using this formula.

$$ ICF = C \times (S. L. Y_i)$$

Where ICF is the Indirect Carbon Footprint, C is the Characterization Factor matrix for Carbon Footprint, L is the Leontief-Inverse matrix representing the production recipe, and Y is the final demand matrix, which value here is the household expenditure.

In [24]:
#Expenditure has to be converted from Euro to MEuro
Footprint = MRIO_Char[4].dot(MRIO_S.dot(MRIO_L).dot(EXIO_Expenditure))/1e6
print(Footprint)

[5.24224218e+02 4.92452796e+02 1.02931680e+02 4.18730083e+02
 8.15729186e+01 1.66086642e+02 2.13705890e+02 1.75395616e+02
 9.86938175e+01 1.71604387e+02 7.61684404e+01 9.42967417e+01
 0.00000000e+00 2.10963048e+01 9.44700408e+02 1.67797594e+01
 4.63319194e+00 1.49125593e+02 1.24252103e+00 3.45261348e+02
 6.74232972e+00 4.11979610e+02 3.29483660e+01 6.09579158e+01
 8.37175707e+00 6.39588882e+02 1.61574640e+03 5.28880059e+02
 4.99208386e+02 1.09952597e+02 1.73350477e+02 2.08918462e+02
 2.96430305e+01 1.91718287e+00 8.84738500e+01 3.13791254e+01
 1.90790566e+01 2.07694811e+00 5.04878434e+01 3.60088531e+01
 1.05695760e+02 1.21871656e+01 2.70702967e+02 5.19231812e+01
 7.78041838e+01 5.06711303e+01 2.94459369e+01 6.90476139e+00
 1.51403637e+01 2.26359067e+02 6.61352635e+00 2.35486286e+01
 0.00000000e+00 1.16878630e+01 1.18519500e+03 7.81236065e+01
 1.85481273e+02 2.40132259e+01 2.96108586e+00 1.46427615e+02
 3.60166267e+01 1.03110327e+02 2.12819219e+00 2.72153256e+00
 2.78174377e+01 5.378019

Since the footprint calculated before only considers the embodied emissions from the production stage and excludes the direct emissions from heating and transportation, household direct emissions need to be calculated additionally by multiplying the emissions intensity (in kg CO<sub>2</sub>/Euro) with the expenditure.

The emissions intensity data for German households are available from Hardadi, et al. (2020). To calculate the emissions intensity for other countries, divide the emissions intensity in physical unit by the fuel prices. The emission intensities in physical unit are:

Heating oil:	11.8 kg CO<sub>2</sub>/gallon

Heating gas:	55.9 ton CO<sub>2</sub>/TJ

Gasoline fuel:	3.18 ton CO<sub>2</sub>/ton


In [25]:
DirEmis_Fuel = 1.68 * CES_DEexpenditure[54] #Expenditure of transportation fuel
DirEmis_Wood = 1.39 * CES_DEexpenditure[29] #Expenditure of solid heating fuel (wood or coal)
DirEmis_HOil = 4.24 * CES_DEexpenditure[28] #Expenditure of heating oil
DirEmis_HGas = 3.35 * CES_DEexpenditure[27] #Expenditure of heating gas
Dir_Emission = DirEmis_Fuel + DirEmis_Wood + DirEmis_HOil + DirEmis_HGas

print(DirEmis_Fuel)
print(DirEmis_Wood)
print(DirEmis_HOil)
print(DirEmis_HGas)
print(Dir_Emission)

1921.9199999999998
48.65
1496.72
1427.1000000000001
4894.39


Finally, the total household carbon footprint is calculated as the sum of indirect carbon footprint and direct emissions.

In [28]:
HH_Footprint = Footprint.sum(axis=0) + Dir_Emission
print(round(HH_Footprint, 2), " kg CO2e/household")

18287.63  kg CO2e/household


## References:

Hardadi, G., Buchholz, A., Pauliuk, S. (2020). Implications of the Distribution of German Household Environmental Footprints across Income Groups for Integrating Environmental and Social Policy Design. Journal of Industrial Ecology, 00(0), 1–11. https://doi.org/10.1111/jiec.12670

Miller, R. E., & Blair, P. D. (2009). Input-Output Analysis. Cambridge University Press.

Min, J., & Rao, N. D. (2017). Estimating Uncertainty in Household Energy Footprints. Journal of Industrial Ecology, 00(0), 1–11. https://doi.org/10.1111/jiec.12670

